In [683]:
import numpy as np
import os
import matplotlib.pyplot as plt
from astropy import units as u
import pandas as pd
from collections import OrderedDict
from scipy.interpolate import interp1d

In [684]:
from lsst.geom import Point2D, Point2I
from lsst.afw.cameraGeom import FIELD_ANGLE, PIXELS
import lsst.afw.cameraGeom.utils as cgUtils
from lsst.daf.persistence import Butler, NoResults

In [685]:
import lsst.syseng.throughputs as st
from lsst.sims.photUtils import PhotometricParameters, Bandpass, LSSTdefaults
from lsst.sims.utils import angularSeparation

In [686]:
#rname = 'R10'
#rname = 'R22'
rname = 'R01'
#rname = 'R11'
#rname = 'R20'
#rname = 'R21'
#rname = 'R30'
#rname = 'R12'
#rname = 'R02'
#rname = 'R31'
#rname = 'R03'
#rname = 'R34'
#rname = 'R13'
#rname = 'R23'
#rname = 'R14'
#rname = 'R32'
#rname = 'R24'
#rname = 'R41'
#rname = 'R42'
#rname = 'R33'
#rname = 'R43'

In [687]:
# mapping based on 
#https://confluence.slac.stanford.edu/pages/viewpage.action?spaceKey=LSSTCAM&title=Raft+Delivery+and+Acceptance+Testing+Status
dd = pd.read_csv('raftInstall.csv',index_col=0)
useDefault = False
try:
    if np.isnan(dd.rtm[rname]):
        useDefault = True
        print('no data yet, use default')
except TypeError:
    pass

### Default photometric parameters, as used in standard m5 calculations
Note that effarea is not in this list here, because it varies with field.

The read noise is not in this list either, because it varies by amp.

In [688]:
exptime=15 
nexp=2
othernoise=0 
darkcurrent=0.2
X=1.0

In [689]:
#we do not need this cell to calculate m5, but these FWHMeff are the default values we use actually
lsstDefaults = LSSTdefaults()

### Set up throughputs for hardware and atmosphere. Use the default detector QE as in syseng_throughput for now

In [690]:
# Add losses to each component?
addLosses = True
defaultDirs = st.setDefaultDirs()
defaultDirs

{'detector': '/home/bxin/notebooks/syseng_throughputs/components/camera/detector/joint_minimum',
 'lens1': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens1',
 'lens2': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens2',
 'lens3': '/home/bxin/notebooks/syseng_throughputs/components/camera/lens3',
 'filters': '/home/bxin/notebooks/syseng_throughputs/components/camera/filters',
 'mirror1': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror1',
 'mirror2': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror2',
 'mirror3': '/home/bxin/notebooks/syseng_throughputs/components/telescope/mirror3',
 'atmosphere': '/home/bxin/notebooks/syseng_throughputs/siteProperties'}

In [691]:
atmos = st.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
mirror1 = st.buildMirror(defaultDirs['mirror1'], addLosses)
mirror2 = st.buildMirror(defaultDirs['mirror2'], addLosses)
mirror3 = st.buildMirror(defaultDirs['mirror3'], addLosses)
lens1 = st.buildLens(defaultDirs['lens1'], addLosses)
lens2 = st.buildLens(defaultDirs['lens2'], addLosses)
lens3 = st.buildLens(defaultDirs['lens3'], addLosses)
filters = st.buildFilters(defaultDirs['filters'], addLosses)

vendor = dd.vendor[rname]
vendorDir = defaultDirs['detector']+'/../'+vendor.lower()
addLosses = False 
detector0 = st.buildDetector(vendorDir, addLosses) #design QE from this vendor
detector = Bandpass()

In [692]:
for f in filters:
    print(f, lsstDefaults.FWHMeff(f), lsstDefaults.m5(f))

r 0.83 24.43
i 0.8 24.0
u 0.92 23.68
z 0.78 24.45
g 0.87 24.89
y 0.76 22.6


### Butler access to the QE data

In [693]:
DATADIR = f"{os.environ['OBS_LSST_DIR']}/lsstcam/CALIB" 
print(DATADIR)
butler = Butler(DATADIR)
cam = butler.get('camera')

#This is no longer needed after tickets/DM-22605
#from lsst.obs.lsst.lsstCamMapper import LsstCamMapper
#mapper = LsstCamMapper()
#lsstcam = mapper.camera

/home/bxin/lsst_stack/obs_lsst/lsstcam/CALIB


### Prepare vignetting function

In [694]:
# below we use v3.11 values
vfile = f"{os.environ['HOME']}/notebooks/f_factors/data/vignettingF.txt"
M1D = 8.36 #clear aperture as in Optical design
aa = np.loadtxt(vfile, skiprows=12)
vr = aa[:,0]
vv = aa[:,1]

### Create the dataframes for this raft

In [695]:
#filterlist = tuple([s for s in filters]+['fS']) # this would be in different order than below
filterlist = ['u', 'g', 'r', 'i', 'z', 'y', 'fS']
alist = ('raDeg', 'decDeg', 'radDeg', 'effarea', 'readnoise')
detectors = []
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    detectors.append(det.getName())
adf = pd.DataFrame(index=alist, columns=detectors, dtype=object)
m5df = pd.DataFrame(index=filterlist, columns=detectors, dtype=object)
Tdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)
Sdf = pd.DataFrame(index=filterlist[:6], columns=detectors, dtype=object)

In [696]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
    key = rname+'_'+dname
    raDeg = {}
    decDeg = {}
    readnoise = {}
    for amp in det:
        i = amp.getName()
        amp_point = amp.getBBox().getCenter()
        raDec = det.transform(amp_point, PIXELS, FIELD_ANGLE) 
        [raDeg[i], decDeg[i]] = np.degrees(raDec)
        #print(key, i, raDec)
        if useDefault:
            readnoise[i] = 8.8
        else:
            readnoise[i] = amp.getReadNoise()
    adf[key].loc['raDeg'] = list(OrderedDict(sorted(raDeg.items())).values())
    adf[key].loc['decDeg'] = list(OrderedDict(sorted(decDeg.items())).values())
    adf[key].loc['readnoise'] = list(OrderedDict(sorted(readnoise.items())).values())
        
    #effetive area
    radius = angularSeparation(0., 0., adf[key]['raDeg'], adf[key]['decDeg'])
    adf[key].loc['radDeg'] = list(radius)
    adf[key].loc['effarea'] = list(np.interp(radius, vr, vv)*np.pi*(M1D/2)**2)

In [697]:
adf

,R01_S00,R01_S01,R01_S02,R01_S10,R01_S11,R01_S12,R01_S20,R01_S21,R01_S22
raDeg,"[-1.0433055555555557, -1.0150277777777779, -0....","[-0.8084166666666668, -0.7801388888888889, -0....","[-0.5735833333333332, -0.5453055555555555, -0....","[-1.0430277777777777, -1.01475, -0.98647222222...","[-0.8079166666666667, -0.779638888888889, -0.7...","[-0.5730277777777777, -0.54475, -0.51647222222...","[-1.0423055555555556, -1.0140277777777778, -0....","[-0.8073611111111112, -0.7790833333333333, -0....","[-0.5725833333333332, -0.5443055555555555, -0...."
decDeg,"[-1.6999999999999997, -1.6999999999999997, -1....","[-1.7007222222222225, -1.7007222222222222, -1....","[-1.7010000000000003, -1.701, -1.701, -1.70100...","[-1.465166666666667, -1.4651666666666665, -1.4...","[-1.4656111111111112, -1.4656111111111112, -1....","[-1.4661666666666666, -1.4661666666666666, -1....","[-1.2302777777777776, -1.2302777777777778, -1....","[-1.2307222222222223, -1.230722222222222, -1.2...","[-1.2311666666666665, -1.2311666666666665, -1...."
radDeg,"[1.9945342920440905, 1.9798936919590506, 1.965...","[1.8830299062810565, 1.8710676100043133, 1.859...","[1.7950772062900082, 1.7862451627317073, 1.777...","[1.7984390881449177, 1.7821914756853439, 1.766...","[1.6735001929799782, 1.6600361877925487, 1.646...","[1.5741454356615305, 1.5640755993980777, 1.554...","[1.6123948976875337, 1.5942640014878735, 1.576...","[1.4718725816044054, 1.4565557569167789, 1.441...","[1.3577822259635128, 1.3461033758651069, 1.334..."
effarea,"[12.22555535777579, 13.822514271377216, 15.242...","[22.008646126933254, 22.73348434630299, 23.458...","[28.491603898496987, 29.094139003830076, 29.63...","[28.262251334258163, 29.370687543853116, 30.22...","[31.527613077024, 31.584113824881776, 31.65734...","[32.08585415601281, 32.14220651167448, 32.1960...","[31.83872001809569, 31.968256568158317, 32.073...","[32.61481196254603, 32.688089260484524, 32.771...","[33.08315545829495, 33.12812621504663, 33.1569..."
readnoise,"[6.62282, 6.55378, 6.72528, 6.8305, 6.79864, 6...","[6.5041, 6.67796, 6.47256, 6.50954, 6.56605, 6...","[7.08046, 7.03323, 7.21267, 7.25466, 7.35687, ...","[5.85558, 6.1201, 6.11318, 6.20205, 6.28053, 6...","[5.64874, 5.93544, 5.79023, 5.82116, 5.85858, ...","[5.71568, 5.89189, 6.01145, 6.12401, 6.1578, 6...","[7.1738, 7.19533, 7.31546, 7.46645, 7.55057, 7...","[11.4992, 12.035, 13.1089, 13.0867, 13.0443, 1...","[6.94761, 7.07155, 7.29113, 7.28697, 7.616, 7...."


In [698]:
ampList = list(OrderedDict(sorted(raDeg.items())).keys())

In [699]:
# this is needed if there are only 6 QE measurements per amp
idx = np.where(detector0.sb>0.01)
idx1=idx[0][0]-1
idx2=idx[0][-1]+1

x1 = detector0.wavelen[idx1]
y1 = detector0.sb[idx1]
x2 = detector0.wavelen[idx2]
y2 = detector0.sb[idx2]

In [700]:
m5SRD = np.array([23.9, 25.0, 24.7, 24.0, 23.3, 22.1])
#m5SRDmin = []
# Nv1 from SRD table 24
Nv1 = np.array([56, 80, 184, 184, 160, 160])
omega = Nv1/sum(Nv1)
fidx = 'ugrizy' #very important!

In [701]:
for det in cam:
    rname1, dname = det.getName().split('_')
    if rname1 != rname: 
        continue;
        
    vendor = det.getSerial()[:3].lower()
    assert useDefault or dd.vendor[rname].lower() == vendor
    vendorDir = defaultDirs['detector']+'/../'+vendor
    print('Calculating m5 for %s_%s'%(rname,dname))
    
    key = rname+'_'+dname
    for f in filterlist:
        m5df[key][f] = [-1.]*len(ampList)
    for f in filterlist[:6]:
        Tdf[key][f] = [-1.]*len(ampList)
        Sdf[key][f] = [-1.]*len(ampList)
        
    for amp in det:
        
        try:
            qe_curve = butler.get('qe_curve', raftName=rname, detectorName=dname, calibDate='1970-01-01T00:00:00')
            wavelen = detector0.wavelen 

            k = amp.getName()
            if len(qe_curve.data[k][0])>10:
                wlen = qe_curve.data[k][0]
                eff = qe_curve.data[k][1]
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='quadratic')
            else:
                aa = np.append(x1, qe_curve.data[k][0].value)
                aa = np.append(aa, x2)
                wlen = aa * qe_curve.data[k][0].unit

                aa = np.append(y1, qe_curve.data[k][1].value)
                aa = np.append(aa, y2)
                eff = aa * qe_curve.data[k][1].unit
                f = interp1d(wlen.value, eff.value, fill_value=0, bounds_error=False, kind='slinear')#quadratic causes overshoot
            
            sb = f(wavelen)*0.01
            #alternatively we could do (only for >10 QE measurements)
            #amp_point = amp.getBBox().getCenter()
            #sb = qe_curve.evaluate(det, amp_point, wavelen* u.nm, kind='quadratic').value*.01 #unit was percent in CALIB data

            sb[np.isnan(sb)] = 0
            if np.max(sb)>1.5:
                print('These seem too LARGE ', k)
                print(np.max(sb))
                sb = 0
            if np.max(sb)<0.2: #3 dead channels, 1 out of each of R01, R10, and R30; see camera confluence page table
                print('dead channel: %s, max sb = %.2f'%(key, np.max(sb)))
                continue;
                
            detector.setBandpass(wavelen, sb)
                
            #detector losses  
            #os.listdir(vendorDir)
            detLosses = Bandpass()
            detLosses.readThroughput(os.path.join(vendorDir, '%s_Losses/det_Losses.dat' % (vendor)))
                
            #build hardware and system
            hardware = {}
            system = {}
            for f in filters:
                sb = mirror1.sb * mirror2.sb *mirror3.sb
                sb *= lens1.sb * lens2.sb * lens3.sb * filters[f].sb
                sb *= detector.sb * detLosses.sb
                
                hardware[f] = Bandpass()
                hardware[f].setBandpass(wavelen, sb)
                system[f] = Bandpass()
                system[f].setBandpass(wavelen, sb * atmos.sb)
                
        except NoResults:
            if not useDefault:
                print('No results found for this detector')
            assert useDefault
            
            hardware, system = st.buildHardwareAndSystem(defaultDirs)
            
        #calculate m5      
        iamp = ampList.index(amp.getName())
        effarea = adf[key]['effarea'][iamp]*100**2 #convert to cm^2
        readnoise = adf[key]['readnoise'][iamp]
        
        m5 = st.makeM5(hardware, system, darksky=None, 
                    exptime=15, nexp=2, readnoise=readnoise, othernoise=0, darkcurrent=0.2,
                    effarea=effarea, X=1.0)
        for f in filters:
            m5df[key][f][iamp] = m5.m5[f]
            Tdf[key][f][iamp] = m5.Tb[f]
            Sdf[key][f][iamp] = m5.Sb[f]
        m5amp = np.array([m5.m5[f] for f in fidx])
        if np.all(m5amp>0):
            m5df[key]['fS'][iamp] = sum(omega*10**(0.8*(m5amp - m5SRD)))
        

Calculating m5 for R01_S00
Calculating m5 for R01_S01
dead channel: R01_S01, max sb = 0.11
Calculating m5 for R01_S02
Calculating m5 for R01_S10
Calculating m5 for R01_S11
Calculating m5 for R01_S12
Calculating m5 for R01_S20
Calculating m5 for R01_S21
Calculating m5 for R01_S22


In [702]:
Sdf

,R01_S00,R01_S01,R01_S02,R01_S10,R01_S11,R01_S12,R01_S20,R01_S21,R01_S22
u,"[0.05945627014087335, 0.0590975551021219, 0.05...","[0.055331539741129455, 0.0551346447248619, 0.0...","[0.059895010633873154, 0.059816977455687995, 0...","[0.06003473477222531, 0.06003364111884467, 0.0...","[0.059452153097014036, 0.060972821354569794, 0...","[0.06625157675622809, 0.06603857678319228, 0.0...","[0.058649411365279784, 0.05827478634466608, 0....","[0.0551634794248291, 0.054724616284364064, 0.0...","[0.0550416037559658, 0.05537525098309908, 0.05..."
g,"[0.15375943906920286, 0.15367983732559984, 0.1...","[0.15727882571642726, 0.15742233421096866, 0.1...","[0.15495359542000867, 0.15558045876181234, 0.1...","[0.15628853666589282, 0.15608643936092856, 0.1...","[0.1543058062031544, 0.15434122185134996, 0.15...","[0.15784051672365393, 0.15756558259885897, 0.1...","[0.1519207523942954, 0.15165974447039535, 0.15...","[0.15295149111069703, 0.15336247933142985, 0.1...","[0.1573104824018438, 0.15758164427583118, 0.15..."
r,"[0.11952088713144783, 0.11936216604004242, 0.1...","[0.11791711587793516, 0.1179041781779769, 0.11...","[0.11969390699393216, 0.11998386146742407, 0.1...","[0.1196457219043775, 0.11936378922524132, 0.11...","[0.11953433588882076, 0.11925424846256492, 0.1...","[0.12344388990604944, 0.1231004678914194, 0.12...","[0.1184982353520735, 0.11810163959381857, 0.11...","[0.11766467093829909, 0.11791037880920292, 0.1...","[0.1177919074105292, 0.11808157231199566, 0.11..."
i,"[0.0889662887264008, 0.08906442655708983, 0.08...","[0.08812195885908834, 0.08811404537615, 0.0879...","[0.08888846207650813, 0.08923564909636827, 0.0...","[0.08839353252623065, 0.08836391440116233, 0.0...","[0.08846582617937414, 0.08842430588174469, 0.0...","[0.08951310573120662, 0.08941381867145354, 0.0...","[0.0887710072494265, 0.08859487225321724, 0.08...","[0.08881059478428761, 0.08905872198053497, 0.0...","[0.08876250471656742, 0.0890510355003244, 0.08..."
z,"[0.06037546435564618, 0.06055360217789476, 0.0...","[0.061136268018376315, 0.06115273758249158, 0....","[0.060092738667142764, 0.060354829485669524, 0...","[0.06014743308173177, 0.06015292993384528, 0.0...","[0.06052264183424179, 0.060431912343882475, 0....","[0.059850342511359206, 0.059803713362739215, 0...","[0.06055809767775061, 0.060541801884841324, 0....","[0.061084782494313074, 0.0613048415028598, 0.0...","[0.06183623003141501, 0.061996698540872835, 0...."
y,"[0.031680171307392604, 0.03178399716642021, 0....","[0.03201749707411871, 0.03205418262777487, 0.0...","[0.03117408308574932, 0.031218708379064118, 0....","[0.031167990530973074, 0.031289942672055655, 0...","[0.03157277250832405, 0.03164427463530244, 0.0...","[0.030457078221477554, 0.030501140162665537, 0...","[0.031864073448734764, 0.031834354981518105, 0...","[0.030919801444197124, 0.031097034202210005, 0...","[0.03232692714142657, 0.03250023436390564, 0.0..."


In [703]:
m5df

,R01_S00,R01_S01,R01_S02,R01_S10,R01_S11,R01_S12,R01_S20,R01_S21,R01_S22
u,"[23.184467698997345, 23.302971750363866, 23.37...","[23.677448784647545, 23.686718980437114, 23.72...","[23.91889476317664, 23.940211797783903, 23.937...","[24.017743167646856, 24.02778958395215, 24.051...","[24.11458885180656, 24.11339859121886, 24.1325...","[24.20074002067121, 24.187046112742376, 24.176...","[23.989777769049162, 23.986209513288426, 23.97...","[23.62884633310714, 23.586287367202836, 23.506...","[23.98985435852997, 23.986339108249126, 23.964..."
g,"[24.20311957773537, 24.298219524754316, 24.361...","[24.647331496646053, 24.663712080088732, 24.69...","[24.791614646911686, 24.809920482531595, 24.81...","[24.829461682566304, 24.845957398067245, 24.86...","[24.896372118426445, 24.89043235024265, 24.897...","[24.919434738399826, 24.915054275832464, 24.91...","[24.849932570652868, 24.850826882808086, 24.85...","[24.725915978706873, 24.710277872449847, 24.67...","[24.90552969683417, 24.904138165566653, 24.899..."
r,"[23.797683392238635, 23.88100833955091, 23.939...","[24.173485865124576, 24.190031952047313, 24.21...","[24.331765203240742, 24.346984198429688, 24.35...","[24.347035684046737, 24.364619778553404, 24.38...","[24.414471743050186, 24.4100587910011, 24.4137...","[24.443162447447055, 24.440209886569782, 24.43...","[24.39202505854141, 24.392147841856602, 24.393...","[24.31822749756855, 24.30916394433181, 24.2833...","[24.41586995238385, 24.41619911735826, 24.4130..."
i,"[23.387269510881115, 23.467640512237054, 23.52...","[23.74838097600888, 23.76524799046093, 23.7861...","[23.901138696690587, 23.916425706204105, 23.92...","[23.90804838065608, 23.92728387283359, 23.9456...","[23.974422022692806, 23.972409386471355, 23.97...","[23.991659614093017, 23.990202215233747, 23.98...","[23.96500648867844, 23.965785003285507, 23.967...","[23.917875895765818, 23.912170928865933, 23.89...","[23.98904203481973, 23.99036483670985, 23.9887..."
z,"[22.830954529188247, 22.90878483626271, 22.965...","[23.190648318591595, 23.207938410309346, 23.22...","[23.327282609055068, 23.342432583268266, 23.35...","[23.331932441434365, 23.35222937539399, 23.370...","[23.398571919152467, 23.39696176195749, 23.400...","[23.403091966943794, 23.402672241819793, 23.40...","[23.394889805262324, 23.39689662177853, 23.400...","[23.372280622209676, 23.369683476320787, 23.35...","[23.430123342135623, 23.431570938748635, 23.43..."
y,"[21.929059811305663, 22.005256776064247, 22.06...","[22.28151319577467, 22.299373913562214, 22.318...","[22.41080741732931, 22.424002471906874, 22.430...","[22.41403796123599, 22.436459107634693, 22.455...","[22.48390716175028, 22.484659912159586, 22.487...","[22.473018359859424, 22.47392408765494, 22.474...","[22.486347330053256, 22.48795215978492, 22.492...","[22.448486241149183, 22.448110122472244, 22.43...","[22.517451034942972, 22.520512403025286, 22.52..."
fS,"[0.378717102942439, 0.43996336847240436, 0.489...","[0.7509609865776149, 0.7745550807893946, 0.806...","[0.984354141665049, 1.0122256883616463, 1.0248...","[1.0126112646721335, 1.0494588118206039, 1.087...","[1.1527919194027734, 1.1496982106930536, 1.160...","[1.1817883760584202, 1.17728874215457, 1.17426...","[1.116680992203222, 1.1186339196645745, 1.1233...","[0.993111610941038, 0.9831869886874485, 0.9518...","[1.178633066810866, 1.1815488498704099, 1.1776..."


In [704]:
dfDir = os.path.join('m5_output', rname)

In [705]:
if not os.path.exists(dfDir):
    os.mkdir(dfDir)
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
adf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
m5df.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Tdf_%s.csv'%rname)
Tdf.to_csv(dfPath)
dfPath = os.path.join(dfDir, 'Sdf_%s.csv'%rname)
Sdf.to_csv(dfPath)

### Make sure we can read out the dataframes correctly

In [706]:
from ast import literal_eval

In [707]:
dfPath = os.path.join(dfDir, 'adf_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [708]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

raDeg        [-1.0433055555555557, -1.0150277777777779, -0....
decDeg       [-1.6999999999999997, -1.6999999999999997, -1....
radDeg       [1.9945342920440905, 1.9798936919590506, 1.965...
effarea      [12.22555535777579, 13.822514271377216, 15.242...
readnoise    [6.62282, 6.55378, 6.72528, 6.8305, 6.79864, 6...
Name: R01_S00, dtype: object

In [709]:
adf

,R01_S00,R01_S01,R01_S02,R01_S10,R01_S11,R01_S12,R01_S20,R01_S21,R01_S22
raDeg,"[-1.0433055555555557, -1.0150277777777779, -0....","[-0.8084166666666668, -0.7801388888888889, -0....","[-0.5735833333333332, -0.5453055555555555, -0....","[-1.0430277777777777, -1.01475, -0.98647222222...","[-0.8079166666666667, -0.779638888888889, -0.7...","[-0.5730277777777777, -0.54475, -0.51647222222...","[-1.0423055555555556, -1.0140277777777778, -0....","[-0.8073611111111112, -0.7790833333333333, -0....","[-0.5725833333333332, -0.5443055555555555, -0...."
decDeg,"[-1.6999999999999997, -1.6999999999999997, -1....","[-1.7007222222222225, -1.7007222222222222, -1....","[-1.7010000000000003, -1.701, -1.701, -1.70100...","[-1.465166666666667, -1.4651666666666665, -1.4...","[-1.4656111111111112, -1.4656111111111112, -1....","[-1.4661666666666666, -1.4661666666666666, -1....","[-1.2302777777777776, -1.2302777777777778, -1....","[-1.2307222222222223, -1.230722222222222, -1.2...","[-1.2311666666666665, -1.2311666666666665, -1...."
radDeg,"[1.9945342920440905, 1.9798936919590506, 1.965...","[1.8830299062810565, 1.8710676100043133, 1.859...","[1.7950772062900082, 1.7862451627317073, 1.777...","[1.7984390881449177, 1.7821914756853439, 1.766...","[1.6735001929799782, 1.6600361877925487, 1.646...","[1.5741454356615305, 1.5640755993980777, 1.554...","[1.6123948976875337, 1.5942640014878735, 1.576...","[1.4718725816044054, 1.4565557569167789, 1.441...","[1.3577822259635128, 1.3461033758651069, 1.334..."
effarea,"[12.22555535777579, 13.822514271377216, 15.242...","[22.008646126933254, 22.73348434630299, 23.458...","[28.491603898496987, 29.094139003830076, 29.63...","[28.262251334258163, 29.370687543853116, 30.22...","[31.527613077024, 31.584113824881776, 31.65734...","[32.08585415601281, 32.14220651167448, 32.1960...","[31.83872001809569, 31.968256568158317, 32.073...","[32.61481196254603, 32.688089260484524, 32.771...","[33.08315545829495, 33.12812621504663, 33.1569..."
readnoise,"[6.62282, 6.55378, 6.72528, 6.8305, 6.79864, 6...","[6.5041, 6.67796, 6.47256, 6.50954, 6.56605, 6...","[7.08046, 7.03323, 7.21267, 7.25466, 7.35687, ...","[5.85558, 6.1201, 6.11318, 6.20205, 6.28053, 6...","[5.64874, 5.93544, 5.79023, 5.82116, 5.85858, ...","[5.71568, 5.89189, 6.01145, 6.12401, 6.1578, 6...","[7.1738, 7.19533, 7.31546, 7.46645, 7.55057, 7...","[11.4992, 12.035, 13.1089, 13.0867, 13.0443, 1...","[6.94761, 7.07155, 7.29113, 7.28697, 7.616, 7...."


In [710]:
dfPath = os.path.join(dfDir, 'm5df_%s.csv'%rname)
df = pd.read_csv(dfPath, index_col=0)

In [711]:
df['%s_S00'%rname].apply(literal_eval)#['readnoise'][0]

u     [23.184467698997345, 23.302971750363866, 23.37...
g     [24.20311957773537, 24.298219524754316, 24.361...
r     [23.797683392238635, 23.88100833955091, 23.939...
i     [23.387269510881115, 23.467640512237054, 23.52...
z     [22.830954529188247, 22.90878483626271, 22.965...
y     [21.929059811305663, 22.005256776064247, 22.06...
fS    [0.378717102942439, 0.43996336847240436, 0.489...
Name: R01_S00, dtype: object

In [713]:
#df['R01_S11']['u']

'[24.11458885180656, 24.11339859121886, 24.13250005620054, 24.12891922372829, 24.124089794076895, 24.130942988895832, 24.134569555523523, 24.159295508368977, 24.160422046091206, 24.148280079772235, 24.130634407365797, 24.139356489514352, 24.144550705752692, 24.151070005094525, 24.144918692503822, 24.144079129360527]'